# Auto-Transformer

## Input

### User Input

- Input Voltage $V_{in}$
- Output Power $P_o=250$
- Frequency $f=47$
- Efficiency $\eta=95\%$
- Operating flux density $B_{ac}$
- Temperature rise goal, $T_r=30^oC$
- Window utilization, $K_u=0.4$
- Core Material = $Silicon M6X$
- Lamination Number = $EI-150$
- Current Density $J=250 [amps/cm^2]$

### Constants 

- Window utilization $K_u$

In [1]:
Frequency = 47 # Hz
Temperature_rise_goal = 30 # degree Celcius
Output_power = 250 # watts
Efficiency = 95 # %
Input_voltage = 115
Bobbin_thickness = 1.5 


## Apparent Power $P_t$

Calculating the transformer apparent power, $P_t$

$$ P_t=P_o(\dfrac{1}{\eta} + 1), [watts]$$

$$P_t=250[(1/0.95)+1]$$

$$P_t=513, [watts]$$

In [2]:
Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
print(Apparent_power)

513.1578947368421


## Calculation of Area Product

$$J=\dfrac{P_t (10^4)}{K_f K_u B_{ac} f A_p} $$

$$A_p=\dfrac{P_t (10^4)}{K_f K_u B_{ac} f J} $$

For initial case :

$$K_f = 4.44$$
$$K_u = 0.4 $$
$$B_{ac} = 1.6$$
$$f= 47$$
$$J=250$$

In [3]:
K_f = 4.44
K_u = 0.4
B_ac = 1.6
J = 250
Area_product = (Apparent_power*(10**4))/(K_f * K_u * B_ac * J * Frequency)
print(Area_product)

153.6916255561272


## Select lamination and vary the area product

In [4]:
import pandas as pd
import numpy as np

In [5]:
lamination_data = pd.read_csv('./EI-Laminations - Sheet1.csv')
lamination_data.head()

,Type,A,B,C,D,E,Tongue,Winding-width,Winding-length,K-ratio,Remarks,Fig-No
0,SB1R,8.00,28.00,25.00,4.00,4.00,8.00,6.000,17.00,816.000000,-,1
1,SB3R,9.50,35.00,29.50,5.00,5.00,9.50,7.750,19.50,1435.687500,-,1
2,17,12.70,38.10,31.75,6.35,6.35,12.70,6.350,19.05,1536.287250,-,1
3,74,18.07,53.93,45.11,8.94,8.94,18.07,8.990,27.23,4423.494439,WW 8.9,2
4,12A,15.87,47.62,38.68,7.93,7.93,15.87,7.945,22.82,2877.308763,-,1


**For looping into each lamination**

    for lamination in lamination_data['Type']:
        selected_lamination = lamination_data[lamination_data['Type'] == lamination]
        print(selected_lamination['K-ratio'].max())

**For looping the Area product varying form 60% to 140% of Area Product**

    for x in range(60, 140, 5):
        present_Area_product = x*Area_product*0.01
        print(str(present_Area_product) + ', original AP: ' + str(Area_product))

In [10]:
stack_data = []
Awg = []

for lamination in lamination_data['Type']:
    selected_lamination = lamination_data[lamination_data['Type'] == lamination]
    # print(selected_lamination)
    for x in range(60, 141, 5):
        present_Area_product = x*Area_product*0.01 * 10**4
        # Calculate stack for the present area product
        stack = present_Area_product / selected_lamination['K-ratio'].max()
        # print('stack : ' + str(stack))
        # print('lamination :' + str(selected_lamination['Type'].max()) + '\nArea product: ' + str(present_Area_product) )
        # print(5* selected_lamination['Tongue'].max())
        if stack < 5 * selected_lamination['Tongue'].max():
            # print('stack : ' + str(stack) + ' tongue: ' + str(selected_lamination['Tongue'].max()))
            # print(stack)
            if stack < 5:
                stack = 5.0
            elif stack%5 == 0:
                stack = stack
            elif stack%5 <= 2.5:
                stack = stack - stack % 5
            elif stack%5 > 2.5:
                stack = stack - stack % 5 + 5
            table_data_stack_and_tongue = {
                'Stack': stack,
                'Tongue': selected_lamination['Tongue'].max()
            }
            stack_data.append(table_data_stack_and_tongue)

            # Primary turns
            # we need core area A_c 
            # A_c = tongue * stack
            A_c = stack * selected_lamination['Tongue'].max()
            # print('A_c = ' + str(A_c))

            # Now primary_turns = (Input_voltage) / (K_f* B_ac* Frequency * A_c)
            primary_turns = (Input_voltage) / (K_f* B_ac* Frequency * A_c)
            # print(primary_turns)

            # Input current I_input
            # Input_current = Apparent_power / (Input_voltage*Efficiency)
            Input_current = Apparent_power / (Input_voltage*Efficiency)
            # print(str(Input_current) + ' [amps]')

            # Primary bare wire area A_wp
            # A_wp = Input_current / Current_density (J)
            A_wp = Input_current / J
            # print(A_wp) # bare area in sqcm so convert it into sqmm


            # Select wire from Wire Table 
            # Table is data provide in (EMD-wire-table.csv)
            Awg_data = pd.read_csv('EMD-wire-table.csv')
            Area_required = A_wp
            required_awg = Awg_data.iloc[(Awg_data['BA-sqcm'] - Area_required).abs().argsort()[:1]]
            # Awg.append({'SWG': required_awg['SWG'], 
            # 'Normal Area': required_awg['Normal Conductor Area mm²'],
            # 'CR Normal': required_awg['CR Normal']
            # })
            selected_awg = required_awg['AWG'].to_string(index=False)
            


            # print(Area_of_selected_awg)

            ## TPL (turns per layer TLP_primary)
            # TPL_primary = wl / diameter_of_wire_with_insulation








# stack_data
# df = pd.DataFrame(stack_data)
# df